In [ ]:
#1 - No, scaling the input features won't do anything
#2 - Yes bc it will constrain the model and regularize it
#3 - B
#4 - D

In [1]:
import boto3 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score
from sklearn.preprocessing import MinMaxScaler

#defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data445-ashlyn-bucket'
bucket = s3.Bucket(bucket_name)

#defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
#input and target variables
X = heart[['age', 'male','education', 'currentSmoker', 'cigsPerDay','BPMeds','prevalentStroke', 'prevalentStroke','prevalentHyp', 'diabetes', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

## splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = .2, stratify = Y)

In [3]:
#transforming inputs to 0-1 scale
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [8]:
#random forest
md_rf = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

## extracting feature importance
md_rf.feature_importances_

array([0.31240832, 0.08533402, 0.30940227, 0.12997055, 0.16288484])

In [5]:
variable_importances = list()

for i in range(0, 100):
    
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = .2, stratify = Y)
   
    ## randomforest
    rf = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)
    
    variable_importances.append(rf.feature_importances_)

variable_importances = pd.DataFrame(variable_importances)

variable_importances.apply(np.mean, axis = 0)

0     0.256283
1     0.029569
2     0.014801
3     0.003146
4     0.026454
5     0.009316
6     0.002397
7     0.002422
8     0.097959
9     0.014825
10    0.040913
11    0.221137
12    0.115184
13    0.042512
14    0.017055
15    0.106028
dtype: float64

In [6]:
#top 5 important variables
X_train = X_train[['age', 'prevalentHyp', 'sysBP', 'diaBP', 'glucose']]
X_test = X_test[['age', 'prevalentHyp', 'sysBP', 'diaBP', 'glucose']]

In [7]:
#transforming inputs to 0-1 scale
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [10]:
md1_top5 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

#predicting
pred1 = md1_top5.predict_proba(X_test)[:, 1]
pred1 = np.where(pred1 < .1, 0, 1)

#recall score
recall_score(Y_test, pred1)

0.875

In [13]:
md2_top5 = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)

#predicting
pred2 = md2_top5.predict_proba(X_test)[:, 1]
pred2 = np.where(pred2 < .1, 0, 1)

#recall score
recall_score(Y_test, pred2)

0.8660714285714286

In [14]:
md3_top5 = RandomForestClassifier(n_estimators = 500, max_depth = 7).fit(X_train, Y_train)

#predicting
pred3 = md3_top5.predict_proba(X_test)[:, 1]
pred3 = np.where(pred3 < .1, 0, 1)

#recall score
recall_score(Y_test, pred3)

0.8660714285714286

In [17]:
final_md1 = []
final_md2 = []
final_md3 = []

#repeating 100 times
for i in range(0, 100):
    
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = .2, stratify = Y)
    
    md1_top5 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)
    md2_top5 = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)
    md3_top5 = RandomForestClassifier(n_estimators = 500, max_depth = 7).fit(X_train, Y_train)
    
    pred1 = md1_top5.predict_proba(X_test)[:, 1]
    pred1 = np.where(pred1 < .1, 0, 1)

    pred2 = md2_top5.predict_proba(X_test)[:, 1]
    pred2 = np.where(pred2 < .1, 0, 1)

    pred3 = md3_top5.predict_proba(X_test)[:, 1]
    pred3 = np.where(pred3 < .1, 0, 1)
   
    final_md1.append(recall_score(Y_test, pred1))
    final_md2.append(recall_score(Y_test, pred2))
    final_md3.append(recall_score(Y_test, pred3))

In [19]:
print('Model 1 avg recall score is:', np.mean(final_md1))
print('Model 2 avg recall score is:', np.mean(final_md2))
print('Model 3 avg recall score is:', np.mean(final_md3))

Model 1 avg recall score is: 0.883125
Model 2 avg recall score is: 0.8560714285714285
Model 3 avg recall score is: 0.8358035714285715


In [ ]:
#I would choose model 1 bc the avg recall score is the highest